In [1]:
import pandas as pd
import numpy as np

# Generación de datos aleatorios

Definimos parámetros para los datos, estos parámetros se deben cambiar en relación a la cantidad de talleres y cantidad de estudiantes por sala.

In [2]:
MAX_AAL_B = 4*70
MAX_AAL_M = 6*70
MAX_ESB_B = 1*70
MAX_ESB_M = 1*70
MAX_CPH_B = 2*50
MAX_CPH_M = 2*50

CANT_ESTUDIANTES = 1040

In [3]:
df = pd.read_csv("datos_arreglados.csv", sep=",", encoding="latin1")
df = df[df["ID"] > 60] #esto pues las primeras filas estaban falladas

Fijamos la semilla pero se puede eliminar para generar muestras diferentes y no siempre la misma.

In [4]:
np.random.seed(639)

Ahora, fijamos el número de datos que queremos crear

In [5]:
n_sint = CANT_ESTUDIANTES

Creamos un dataframe donde tendremos los nuevos datos, estos datos serán generados con diferentes combinaciones de "categorías"

In [6]:
df_sint = pd.DataFrame()

# 2.1. ID nuevo (que no coincida con el original)
df_sint["ID"] = range(1, n_sint + 1)

In [7]:
print(df["registration_started"].head(20))

11    2/5/25 10:15:03
12    2/5/25 10:21:33
13    2/5/25 10:30:21
14    2/5/25 10:47:07
15    2/5/25 10:52:21
16    2/5/25 10:54:59
17    2/5/25 11:14:30
18    2/5/25 11:12:36
19    2/5/25 11:46:00
20    2/5/25 12:16:57
21    2/5/25 12:24:04
22    2/5/25 12:30:44
23    2/5/25 10:42:14
24    2/5/25 12:05:59
25    2/5/25 12:44:22
26    2/5/25 12:59:49
27    2/5/25 13:06:37
28    2/5/25 13:57:41
29    2/5/25 13:56:38
30    2/5/25 13:54:15
Name: registration_started, dtype: object


Para generar datos más realistas, los tiempos también se generaran al azar

In [8]:
df.columns = df.columns.str.strip()

# Convertimos a datetime las columnas de tiempo
for col in ["registration_started", "registration_finished"]:
    # Depende del formato del excel, esto cambia
    # df[col] = pd.to_datetime(df[col], format="%d-%m-%Y %H:%M", errors="coerce")
    df[col] = pd.to_datetime(df[col], format="%m/%d/%y %H:%M:%S", errors="coerce")

# Duración real en segundos
duracion_real = (
    df["registration_finished"] - df["registration_started"]
).dt.total_seconds().dropna()

# ----------------- TIEMPOS SINTÉTICOS -----------------
start_base = df["registration_started"].dropna().sample(
    n_sint, replace=True
).reset_index(drop=True)

dur_base = duracion_real.sample(n_sint, replace=True).reset_index(drop=True)

jitter_start = np.random.normal(loc=0, scale=600, size=n_sint)  # ±10 min
jitter_dur   = np.random.normal(loc=0, scale=30,  size=n_sint)  # ±30 s

dur_min = duracion_real.min()
dur_max = duracion_real.max()

dur_sint = dur_base + jitter_dur
dur_sint = np.clip(
    dur_sint,
    a_min=max(1, dur_min * 0.5),
    a_max=dur_max * 1.5
)

start_sint  = start_base + pd.to_timedelta(jitter_start, unit="s")
finish_sint = start_sint + pd.to_timedelta(dur_sint, unit="s")

df_sint["registration_started"]  = start_sint
df_sint["registration_finished"] = finish_sint

Para crear los datos, notemos que los estudiantes de bachelor tienen rellenadas dos columnas que los estudiantes de magister no.

In [9]:
# Quitamos columnas ya tratadas
df_sint["SUMA"] = 6
cols_excluir = ["ID", "registration_started", "registration_finished", "SUMA"]
sin_excluir_cols = [c for c in df.columns if c not in cols_excluir]
select_cols = [col for col in sin_excluir_cols if "Select" in col]
num1_cols = [col for col in sin_excluir_cols if "Select" not in col and "_1" in col]
otras_cols = [col for col in sin_excluir_cols if "Select" not in col and "_1" not in col]

In [10]:
df_bachelor = df[df["program"] == "bachelor"]
df_master = df[df["program"] == "master"]
print("---------------Últimas 2 columnas------------\n")
print(f"Hay {len(df_bachelor)} estudiantes de bachelor y {len(df_master)} estudiantes de master\n")
print("La penúltima columna hay espacios en blanco:\n")
print(f"Bachelor: {df_bachelor[select_cols[-2]].isna().sum()}, Master: {df_master[select_cols[-2]].isna().sum()}")
print("\nLa última columna hay espacios en blanco:\n")
print(f"Bachelor: {df_bachelor[select_cols[-1]].isna().sum()}, Master: {df_master[select_cols[-1]].isna().sum()}")

---------------Últimas 2 columnas------------

Hay 306 estudiantes de bachelor y 468 estudiantes de master

La penúltima columna hay espacios en blanco:

Bachelor: 0, Master: 468

La última columna hay espacios en blanco:

Bachelor: 0, Master: 468


Por ello, importa si son estudiantes de bachelor o master al momento de rellenar las columnas

In [11]:
j = 0
for i in range(len(sin_excluir_cols)):
    col = sin_excluir_cols[i]
    col_data = df[col].dropna()
    col_data_master = df_master[col].dropna()
    col_data_bachelor = df_bachelor[col].dropna()

    if col_data.empty:
        # si la columna está vacía, llenamos con NaN
        df_sint[col] = np.nan
        continue
    
    # Para elegir campus o programa, todos deben tenerlo (i=0 e i=1)
    if i == 0:
        categorias = col_data.unique()
        maximos = {
            ("AAL", "bachelor"): MAX_AAL_B,
            ("AAL", "master"): MAX_AAL_M,
            ("CPH", "bachelor"): MAX_CPH_B,
            ("CPH", "master"): MAX_CPH_M,
            ("ESB", "bachelor"): MAX_ESB_B,
            ("ESB", "master"): MAX_ESB_M
        }
        pool = []
        for (v1, v2), max_uso in maximos.items():
            pool.extend([(v1, v2)] * max_uso)
        pool = np.array(pool, dtype=object)
        np.random.shuffle(pool)
        pares = pool[:n_sint]
        df_pares = pd.DataFrame(pares, columns=["campus", "program"], index=df_sint.index)
        df_sint[["campus", "program"]] = df_pares

    elif i == 1:
        pass
        
        # Para las columnas siguientes todas deben tener hasta 8 elecciones de talleres diferentes
    elif col in select_cols:
        if j == 0:
            df_sint.loc[df_sint["program"]=="bachelor", col] = np.random.choice(
                df_bachelor[col].dropna().unique(),
                size= (df_sint["program"] == "bachelor").sum(),
                replace=True
            )
            df_sint.loc[df_sint["program"]=="master", col] = np.random.choice(
                df_master[col].dropna().unique(),
                size= (df_sint["program"] == "master").sum(),
                replace=True
            )
        elif j < len(select_cols) - 2:
            categorias_bach = df_bachelor[col].dropna().unique()

            for idx in df_sint.index[df_sint["program"] == "bachelor"]:
                # valores ya usados en esta fila, en las columnas anteriores
                usados = set(
                    v for v in df_sint.loc[idx, select_cols[:j]].tolist()
                    if pd.notna(v)
                )

                # categorías disponibles (las que NO están usadas)
                disponibles = [c for c in categorias_bach if c not in usados]

                if disponibles:
                    df_sint.at[idx, col] = np.random.choice(disponibles)
                else:
                    df_sint.at[idx, col] = np.nan

            categorias_master = df_master[col].dropna().unique()

            for idx in df_sint.index[df_sint["program"] == "master"]:
                usados = set(
                    v for v in df_sint.loc[idx, select_cols[:j]].tolist()
                    if pd.notna(v)
                )

                disponibles = [c for c in categorias_master if c not in usados]

                if disponibles:
                    df_sint.at[idx, col] = np.random.choice(disponibles)
                else:
                    df_sint.at[idx, col] = np.nan

        # En las últimas 2 columnas, solo bachelor tiene programas
        else:
            df_sint.loc[(df_sint["program"]=="bachelor"), col] = np.random.choice(
                df_bachelor[col].dropna().unique(),
                size= (df_sint["program"] == "bachelor").sum(),
                replace=True
            )
        j += 1
        
    elif col in otras_cols:
        for a in range(3):
            mask = (df_sint[select_cols[a]].astype(str).str.contains(col, na=False, regex=False))
            df_sint.loc[mask, col] = str(a + 1)
    else:
        df_sint[col] = np.nan

    

# Guardar el csv y que coincidan las columnas
df_sint = df_sint[df.columns]
df_sint.to_csv("datos_nuevos.csv", index=False)
